# Visualize Landprices


In [1]:
import pandas as pd
import geopandas as gpd
import folium

C:\Users\BerkÖztürk\AppData\Local\Temp\ipykernel_3152\2481901238.py:2: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


## Read and explore data 

### Vector data 
We start by reading and quickly visualising vector data

In [ ]:
# read geo files and land prices
prices_koeln = pd.read_csv("../data/raw/1 Land Prices/Land_Prices_Neighborhood_Köln.csv", sep=";", encoding= "utf-8-sig").drop(columns="Unnamed: 0")
neighborhood_koeln = gpd.read_file("../data/raw/3 Neighborhoods/Neighborhoods_Köln.gpkg")
buildings_koeln = gpd.read_file("../data/raw/4 Buildings/Buildings_Köln.gpkg")

In [ ]:
# merge files
prices_koeln = pd.read_csv("../data/raw/1 Land Prices/Land_Prices_Neighborhood_Köln.csv", sep=";", encoding= "utf-8-sig").drop(columns="Unnamed: 0")
neighborhood_koeln = gpd.read_file("../data/raw/3 Neighborhoods/Neighborhoods_Köln.gpkg")
neighborhood_koeln = neighborhood_koeln.merge(prices_koeln, on = "Neighborhood_FID", how = "inner")

### Read raster data

We read the census data that is classified in a grid, then we read the grid as a geopandas file.

In [21]:
import os

In [70]:
ZENSUS_PATH = r"../data/raw/2 Zensus/"
NEIGHBORHOOD_PATH =  r"../data/raw/3 Neighborhoods"
LAND_PRICES_PATH = r"../data/raw/1 Land Prices"
CITY_NAMES=["Berlin","Bremen","Dresden","Frankfurt","Köln"]
zensus_files= os.listdir(ZENSUS_PATH)
neighborhood_files= os.listdir(NEIGHBORHOOD_PATH)
landprices_files= os.listdir(LAND_PRICES_PATH)

city_files={}
for city_name in CITY_NAMES:
    
    csv_fpaths= [os.path.join(ZENSUS_PATH,fpath) for fpath in zensus_files if ( city_name in fpath ) and (fpath.endswith(".csv")) ]
    gpkg_fpaths= [os.path.join(ZENSUS_PATH,fpath) for fpath in zensus_files if ( city_name in fpath ) and (fpath.endswith(".gpkg")) ]
    
    neighbourhood_fpaths= [os.path.join(NEIGHBORHOOD_PATH,fpath) for fpath in neighborhood_files if ( city_name in fpath ) and (fpath.endswith(".gpkg")) ]
    landprices_fpaths= [os.path.join(LAND_PRICES_PATH,fpath) for fpath in landprices_files if ( city_name in fpath ) ]
    print(landprices_fpaths)
    city_files[city_name]= (csv_fpaths,gpkg_fpaths,neighbourhood_fpaths,landprices_fpaths)

['../data/raw/1 Land Prices\\Land_Prices_Neighborhood_Berlin.csv', '../data/raw/1 Land Prices\\Land_Prices_Neighborhood_Berlin.gpkg']
['../data/raw/1 Land Prices\\Land_Prices_Neighborhood_Bremen.csv', '../data/raw/1 Land Prices\\Land_Prices_Neighborhood_Bremen.gpkg']
['../data/raw/1 Land Prices\\Land_Prices_Neighborhood_Dresden.csv', '../data/raw/1 Land Prices\\Land_Prices_Neighborhood_Dresden.gpkg']
['../data/raw/1 Land Prices\\Land_Prices_Neighborhood_Frankfurt_am_Main.csv', '../data/raw/1 Land Prices\\Land_Prices_Neighborhood_Frankfurt_am_Main.gpkg']
['../data/raw/1 Land Prices\\Land_Prices_Neighborhood_Köln.csv', '../data/raw/1 Land Prices\\Land_Prices_Neighborhood_Köln.gpkg']


In [88]:
city_merged_data={}

for city_name, (csv_fpaths,gpkg_fpaths,neighbourhood_fpaths,landprices_fpaths) in city_files.items():
    df_list_city=[]
    for csv_fpath in csv_fpaths:
        df = pd.read_csv(csv_fpath, sep=";", encoding="utf-8-sig").drop(columns="Unnamed: 0")
        df_list_city.append(df)

    from functools import reduce
    zensus_csv_merged = reduce(lambda df1,df2: pd.merge(df1,df2,on="Grid_Code"), df_list_city)
    
    # read grid
    grid_city = gpd.read_file(gpkg_fpaths[0])
    grid_city = grid_city.merge(zensus_csv_merged, on = "Grid_Code")

    # merge files
    prices_city = pd.read_csv(landprices_fpaths[0], sep=";", encoding= "utf-8-sig").drop(columns="Unnamed: 0")
    neighborhood_city = gpd.read_file(neighbourhood_fpaths[0])
    neighborhood_city = neighborhood_city.merge(prices_city, on = "Neighborhood_FID", how = "inner")

    # Perform spatial join using sjoin
    merged_data = gpd.sjoin(grid_city, neighborhood_city, how='left', op='intersects')
    merged_data = merged_data.drop(columns=["City_Name_y", "City_Code_right"]).rename(columns={"City_Name_x":"City_Name", "City_Code_left":"City_Code"})
    city_merged_data[city_name] = merged_data
 

c:\anaconda3\envs\ifohack_spatial_py310\lib\site-packages\IPython\core\interactiveshell.py:3445: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
c:\anaconda3\envs\ifohack_spatial_py310\lib\site-packages\IPython\core\interactiveshell.py:3445: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
c:\anaconda3\envs\ifohack_spatial_py310\lib\site-packages\IPython\core\interactiveshell.py:3445: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
c:\anaconda3\envs\ifohack_spatial_py310\lib\site-packages\IPython\core\interactiveshell.py:3445: FutureWarning: The `op` parameter is deprecated and

In [ ]:
col_list=[]
for city_name,df in city_merged_data.items():
    print(df.shape)
    col_list.append(df.columns.to_list())

In [118]:
common_cols=list(set(col_list[0]).intersection(*col_list))

In [119]:
dfs = [df[common_cols].set_index(["City_Code","Neighborhood_FID"]) for df in city_merged_data.values()]

In [120]:
pd.concat(dfs, axis=1)

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [122]:
df[common_cols].set_index(["City_Code","Neighborhood_FID"]).reset_index()

,City_Code,Neighborhood_FID,we_private_individuals,gg_onefamily_semidetached,re_roman_catholic,gs_buildings_with_living_space,w_30_60,gs_dormitory,e_private_housing_company,al_60_69,...,famk_reg_partner_wout_child,fam_civil_partner_deceased,famk_couple,fams_seniors_young_x,hhleb_single,alk_65_over,r_2,hhtyp_single,gg_apartment_7_12,hhleb_single_fathers
0,05315000,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,05315000,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,05315000,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,05315000,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,05315000,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45325,05315000,81,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
45326,05315000,81,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
45327,05315000,81,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
45328,05315000,81,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [110]:
common_cols

{'Area_Count',
 'Area_Types',
 'City_Code',
 'City_Name',
 'Grid_Code',
 'Land_Value',
 'Neighborhood_FID',
 'Neighborhood_Name',
 'al_10_19',
 'al_10_under',
 'al_20_29',
 'al_30_39',
 'al_40_49',
 'al_50_59',
 'al_60_69',
 'al_70_79',
 'al_80_over',
 'alk_18_29',
 'alk_18_under',
 'alk_30_49',
 'alk_50_64',
 'alk_65_over',
 'buildings_total_units',
 'e_community_of_apartment_owners',
 'e_federal_or_state_government',
 'e_housing_cooperative',
 'e_municipality_housing_company',
 'e_nonprofit_organization',
 'e_other_private_company',
 'e_private_housing_company',
 'e_private_persons',
 'fam_civil_partner',
 'fam_civil_partner_deceased',
 'fam_civil_partner_dissolved',
 'fam_divorced',
 'fam_married',
 'fam_single',
 'fam_unknown',
 'fam_widowed',
 'famgr_2_pers',
 'famgr_3_pers',
 'famgr_4_pers',
 'famgr_5_pers',
 'famgr_6_more',
 'families_total_units',
 'famk_couple',
 'famk_couple_1_child_less_18',
 'famk_couple_children_more_18',
 'famk_couple_wout_child',
 'famk_fathers',
 'famk_

In [65]:
city_merged_files["Bremen"].columns

Index(['Grid_Code', 'City_Code', 'geometry', 'buildings_total_units',
       'n_occupied_by_owner', 'n_ownership_with_current_household',
       'n_owned_without_current_household',
       'n_rented_for_residential_purposes', 'n_rented_with_current_household',
       'n_rented_without_current_household',
       ...
       'sh_russia', 'sh_turkey', 'sh_ukraine', 'sh_other', 'sk_germany',
       'sk_abroad', 'sz_one', 'sz_mult_german_foreign', 'sz_mult_foreign_only',
       'sz_unknown'],
      dtype='object', length=186)

In [47]:
df_lists=[]
for fname_list in fname_lists:
    df_list=[]
    for fname in fname_list:
        df = pd.read_csv(fname, sep=";", encoding="utf-8-sig").drop(columns="Unnamed: 0")
        df_list.append(df)

    from functools import reduce
    df = reduce(lambda df1,df2: pd.merge(df1,df2,on="Grid_Code"), df_list)
    df_lists.append(df)

In [48]:
dfs = [df.set_index('id') for df in df_lists]


KeyError: "None of ['id'] are in the columns"

In [55]:
df_lists[0]

,Grid_Code,buildings_total_units,n_occupied_by_owner,n_ownership_with_current_household,n_owned_without_current_household,n_rented_for_residential_purposes,n_rented_with_current_household,n_rented_without_current_household,n_vacation_and_leisure,n_vacant,...,sh_russia,sh_turkey,sh_ukraine,sh_other,sk_germany,sk_abroad,sz_one,sz_mult_german_foreign,sz_mult_foreign_only,sz_unknown
0,100mN32539E45693,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,100mN32539E45694,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,100mN32539E45695,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,100mN32539E45696,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,100mN32540E45692,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89119,100mN32906E45559,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
89120,100mN32906E45560,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
89121,100mN32906E45561,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
89122,100mN32906E45562,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
print pd.concat(dfs, axis=1)

,Grid_Code,buildings_total_units,n_occupied_by_owner,n_ownership_with_current_household,n_owned_without_current_household,n_rented_for_residential_purposes,n_rented_with_current_household,n_rented_without_current_household,n_vacation_and_leisure,n_vacant,...,sh_russia,sh_turkey,sh_ukraine,sh_other,sk_germany,sk_abroad,sz_one,sz_mult_german_foreign,sz_mult_foreign_only,sz_unknown
0,100mN32539E45693,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,100mN32539E45694,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,100mN32539E45695,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,100mN32539E45696,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,100mN32540E45692,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89119,100mN32906E45559,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
89120,100mN32906E45560,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
89121,100mN32906E45561,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
89122,100mN32906E45562,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [2]:
# Read census data 
buildings_zensus_kl = pd.read_csv("../data/raw/2 Zensus/Zensus_Köln_Buildings.csv", sep=";", encoding="utf-8-sig").drop(columns="Unnamed: 0")
families_zensus_kl = pd.read_csv("../data/raw/2 Zensus/Zensus_Köln_Families.csv", sep=";", encoding="utf-8-sig").drop(columns="Unnamed: 0")
households_zensus_kl = pd.read_csv("../data/raw/2 Zensus/Zensus_Köln_Households.csv", sep=";", encoding="utf-8-sig").drop(columns="Unnamed: 0")
population_zensus_kl = pd.read_csv("../data/raw/2 Zensus/Zensus_Köln_Population.csv", sep=";", encoding="utf-8-sig").drop(columns="Unnamed: 0")

# read grid
grid_koeln = gpd.read_file("../data/raw/2 Zensus/Zensus_Köln_Grid_100m.gpkg")

In [4]:
grid_koeln

,Grid_Code,City_Code,geometry
0,100mN30930E40998,05315000,"POLYGON ((4099800.000 3093000.000, 4099800.000..."
1,100mN30930E40999,05315000,"POLYGON ((4099900.000 3093000.000, 4099900.000..."
2,100mN30931E40997,05315000,"POLYGON ((4099700.000 3093100.000, 4099700.000..."
3,100mN30931E40998,05315000,"POLYGON ((4099800.000 3093100.000, 4099800.000..."
4,100mN30931E40999,05315000,"POLYGON ((4099900.000 3093100.000, 4099900.000..."
...,...,...,...
40508,100mN31128E41001,05315000,"POLYGON ((4100100.000 3112800.000, 4100100.000..."
40509,100mN31128E41002,05315000,"POLYGON ((4100200.000 3112800.000, 4100200.000..."
40510,100mN31128E41003,05315000,"POLYGON ((4100300.000 3112800.000, 4100300.000..."
40511,100mN31128E41004,05315000,"POLYGON ((4100400.000 3112800.000, 4100400.000..."


In [6]:
grid_koeln = grid_koeln.merge(buildings_zensus_kl, on = "Grid_Code")
grid_koeln = grid_koeln.merge(families_zensus_kl, on = "Grid_Code")
grid_koeln = grid_koeln.merge(households_zensus_kl, on = "Grid_Code")
grid_koeln = grid_koeln.merge(population_zensus_kl, on = "Grid_Code")

In [19]:
buildings_zensus_kl.filter(like="n_").sum()

n_occupied_by_owner                        0
n_ownership_with_current_household    136190
n_owned_without_current_household       2504
n_rented_for_residential_purposes          0
n_rented_with_current_household       377613
n_rented_without_current_household      6971
n_vacation_and_leisure                    96
n_vacant                               11684
n_diplomatic_army                          0
n_commercial_use                           0
dtype: int64

In [9]:
grid_koeln.n_rented_for_residential_purposes.value_counts()

n_rented_for_residential_purposes
0    40512
Name: count, dtype: int64

### Spatial merge to neighborhood data

In [ ]:
# Perform spatial join using sjoin
merged_data = gpd.sjoin(grid_koeln, neighborhood_koeln, how='left', op='intersects')
merged_data = merged_data.drop(columns=["City_Name_y", "City_Code_right"]).rename(columns={"City_Name_x":"City_Name", "City_Code_left":"City_Code"})


In [ ]:
# Save merged data to file
merged_data.to_file('../data/interim/merged_data.gpkg', driver='GPKG')